In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from collections import defaultdict
from sklearn.model_selection import StratifiedGroupKFold

categories = {
    1: 'General trash',
    2: 'Paper',
    3: 'Paper pack',
    4: 'Metal',
    5: 'Glass',
    6: 'Plastic',
    7: 'Styrofoam',
    8: 'Plastic bag',
    9: 'Battery',
    10: 'Clothing'
}

data_dir = '/opt/ml/input/data'

In [2]:
# 추출할 json 경로 지정
file_name = 'train_all.json' #########지정해줄꺼 1##########
target_dir = data_dir + '/' + file_name

result = defaultdict(list)
target_json = json.load(open(target_dir))

# annotation과 클래스 추출
for info in target_json['annotations']:
    
    category_id = info['category_id']
    area = info['area']

    result[categories[category_id]].append(area)

# result 값 확인
for k,v in result.items():
    result[k].sort()
    print(k,len(v))

Plastic bag 7643
Plastic 3090
Glass 610
General trash 2782
Metal 562
Paper 9311
Paper pack 659
Styrofoam 1343
Clothing 177
Battery 63


In [3]:
# cut: 클래스별 0.3, 0.6 quantile 계산 후 저장
cut = defaultdict(list)

for k in list(result.keys()):
    cut[k].append(round(pd.Series(result[k]).quantile(.3)))
    cut[k].append(round(pd.Series(result[k]).quantile(.6)))
for k,v in cut.items():
    print(k,v)

Plastic bag [2482, 10254]
Plastic [648, 2429]
Glass [1581, 7892]
General trash [365, 1397]
Metal [965, 6430]
Paper [892, 4066]
Paper pack [560, 2216]
Styrofoam [2210, 6626]
Clothing [3102, 18218]
Battery [4365, 5834]


In [4]:
file_name = 'train_all.json' #########지정해줄꺼 1##########
target_dir = data_dir + '/' + file_name
target_coco = COCO(target_dir)
var = [[target_coco.loadImgs(info['image_id'])[0]['file_name'], info['category_id'], info['area']] for info in target_json['annotations']]

for i, (file_name, cat_id, area) in enumerate(var):
    if area>cut[categories[cat_id]][1]:
        var[i][1] += 20
    elif area>cut[categories[cat_id]][0]:
        var[i][1] += 10

X = np.array([v[0] for v in var])
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=2022)
fold_names = dict()
print()
for i, (train_idx, test_idx) in enumerate(cv.split(X, y, groups)):
    train_names = set(groups[train_idx])
    test_names = set(groups[test_idx])
    fold_names[f'train{i}'] = train_names
    fold_names[f'test{i}'] = test_names
    #print("Train:", len(train_names))
    #print("     :", y[train_idx])
    #print("test:", len(set(groups[test_idx])))
    #print("     :", y[test_idx])

for k,v in fold_names.items():
    print(k,len(v))


loading annotations into memory...
Done (t=4.99s)
creating index...
index created!

train0 2622
test0 649
train1 2616
test1 655
train2 2619
test2 652
train3 2611
test3 660
train4 2616
test4 655


In [5]:
# 원하는 fold 저장

data_dir = '/opt/ml/input/data'

# 변경할 json 경로 지정
file_name = 'train_all_sorted.json'
target_dir = data_dir + '/' + file_name
remove_thr = 16

# 제거할 이미지 파일명 추가: ######### train1저장시 test1 제거, test1저장시 train1 제거 ##########
exclude = set(fold_names['test4'])


target_json = json.load(open(target_dir))
target_coco = COCO(target_dir)
print(f'기존 이미지 개수 = {len(target_json["images"])}')
print(f'기존 annotation 개수 = {len(target_json["annotations"])}')

# 새 json 객체 생성
new_json = {
    'info': target_json['info'],
    'licenses': target_json['licenses'],
    'images': [],
    'categories': target_json['categories'],
    'annotations': [],
}

img_idx = 0 # img 인덱스 -> 순서대로 0부터 1씩 증가해야함
anno_idx = 0 # ann 인덱스 -> 이미지 상관없이 순서대로 0부터 1씩 증가해야함
print(f'시작 img 인덱스 = {img_idx}')
print(f'시작 anno 인덱스 = {anno_idx}')
removed_annos_cnt = 0
small_removed_annos_cnt = 0
for i in range(len(target_json['images'][:])):
    
    img_id = target_coco.getImgIds(imgIds=i)
    img_info = target_coco.loadImgs(img_id)[0]
    img_name = img_info['file_name']
    
    if img_name in exclude:
        ann_ids = target_coco.getAnnIds(imgIds=img_info['id'])
        anns = target_coco.loadAnns(ann_ids)
        removed_annos_cnt += len(anns)
        continue

    ann_ids = target_coco.getAnnIds(imgIds=img_info['id'])
    anns = target_coco.loadAnns(ann_ids)
    anns.sort(key=lambda ann:ann['area'], reverse=True) # area 큰 순

    new_json['images'].append(
        {
            'license':0,
            'url': None,
            'file_name': img_name,
            'height': 512,
            'width': 512,
            'date_captured': None,
            'id': img_idx,
        }
    )
    
    for ann in anns:
        ann['id'] = anno_idx
        ann['image_id'] = img_idx

        if ann['area'] < remove_thr:
            small_removed_annos_cnt += 1
            continue

        anno_idx += 1
        new_json['annotations'].append(ann)

    img_idx += 1

print(f'마지막 img 인덱스 = {img_idx}')
print(f'마지막 anno 인덱스 = {anno_idx}')

print(f'삭제된 이미지 개수 = {len(exclude)}')
print(f'삭제된 annotation 개수 = {removed_annos_cnt}')

print(f'작아서 삭제된 annotation 개수 = {small_removed_annos_cnt}')

print(f'변경 후 이미지 개수 = {len(new_json["images"])}')
print(f'변경 후 annotation 개수 = {len(new_json["annotations"])}')

loading annotations into memory...
Done (t=4.47s)
creating index...
index created!
기존 이미지 개수 = 3272
기존 annotation 개수 = 26240
시작 img 인덱스 = 0
시작 anno 인덱스 = 0
마지막 img 인덱스 = 2617
마지막 anno 인덱스 = 21109
삭제된 이미지 개수 = 655
삭제된 annotation 개수 = 5117
작아서 삭제된 annotation 개수 = 14
변경 후 이미지 개수 = 2617
변경 후 annotation 개수 = 21109


In [55]:
# 저장
save_path = os.path.join(data_dir,f'{file_name.split(".")[0]}_train4.json')

with open(save_path,'w') as f:
        json.dump(new_json, f, indent=4)

# coco 포멧 맞는지 확인
demo = COCO(save_path)

loading annotations into memory...
Done (t=4.44s)
creating index...
index created!


In [ ]:
# fold의 train과 val의 중복되는 이미지 확인 & 총 이미지 수 확인 후 기존 이미지 수와 비교: 3272
train_dir = '/opt/ml/input/data/train_all_sorted_train4.json'
val_dir = '/opt/ml/input/data/train_all_sorted_val4.json'

t_json = json.load(open(train_dir))
v_json = json.load(open(val_dir))
t_names = set([info['file_name'] for info in t_json['images']])
v_names = set([info['file_name'] for info in v_json['images']])

t_names & v_names, len(t_names)+len(v_names), len(json.load(open('/opt/ml/input/data/train_all_sorted.json'))['images'])

In [74]:
# fold0과 나머지 fold의 겹치는 이미지 개수 확인
t0_dir = '/opt/ml/input/data/fold5/train_all_sorted_train0.json'
t1_dir = '/opt/ml/input/data/fold5/train_all_sorted_train1.json'
t2_dir = '/opt/ml/input/data/fold5/train_all_sorted_train2.json'
t3_dir = '/opt/ml/input/data/fold5/train_all_sorted_train3.json'
t4_dir = '/opt/ml/input/data/fold5/train_all_sorted_train4.json'


t0_names = set([info['file_name'] for info in json.load(open(t0_dir))['images']])
t1_names = set([info['file_name'] for info in json.load(open(t1_dir))['images']])
t2_names = set([info['file_name'] for info in json.load(open(t2_dir))['images']])
t3_names = set([info['file_name'] for info in json.load(open(t3_dir))['images']])
t4_names = set([info['file_name'] for info in json.load(open(t4_dir))['images']])

print(len(t0_names & t1_names & t2_names & t3_names & t4_names))
print(len(t0_names & t1_names))
print(len(t0_names & t2_names))
print(len(t0_names & t3_names))
print(len(t0_names & t4_names))

1
1968
1971
1963
1968
